<a href="https://colab.research.google.com/github/koalalovepabro/Deep-Learning/blob/main/10_%EC%97%AD%EC%A0%84%ED%8C%8C_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 계층( Layer )
* 국소적 계산
  * 내가 관심있는 연산만 신경을 쓰는 것
  * **계층**별로 국소적 계산이 일어난다라고 생각하면 된다.
  * 하나의 계층은 하나의 일만(국소적) 전문적으로 할 수 있어야 합니다.
    * 예를 들어 ReLU 계층을 만들었으면, ReLU에 대한 순전파(forward), 역전파(backward)만 신경 쓰자

## 레이어 구현의 주안점
* 순전파와, 역전파를 어떻게 구현할지에 대한 계획
  1. 논문을 보고 구축하는 경우
    * 논문에 나와있는 순전파(with 역전파) 수식을 연구
    * 수식을 효율적, 간편하게 짤 수 있는지에 대한 연구
      * 석, 박사급은 이러한 수식을 연구하고, 만드는 것
      * 우리는 이것들을 구현을 해야 한다.(컴퓨터 시스템에 맞게)
  2. 스스로 비즈니스를 만들어서 하는 경우
    * 비즈니스에 맞게 레이어를 짠다는 이야기는 비즈니스에 맞는 수식을 개발한다.
      * 추가적으로 기존 레이어나 알고리즘들을 커스터마이징 하거나 조합하는 등의 범위도 포함
    * 수식을 세우고 계속 테스트를 해봐야 한다.
      * 수식이 컴퓨터에 효율적인지를 테스트 해본다. - Overflow, Underflow 등 컴퓨터에 맞는 수식을 세웠는지에 대한 검증이 필요

# 1. 곱셈 계층 구현하기
입력이 `x, y`일 때
* `forward`(순전파) : `x * y`
* `backward`(역전파) : `dx = 미분값 * y`, `dy = 미분값 * x`
* 비고 : forward할 때 들어왔던 값들을 저장하고 있어야 한다.

In [ ]:
class MulLayer:
  
  # 딥러닝 레이어의 초기화(생성자에서)는 레이어에서 사용할 옵션이나 변수를 미리 준비
  def __init__(self):
    self.x = None
    self.y = None
  
  # 곱셈레이어의 순전파에서 역전파에 필요한 변수를 저장.
  def forward(self, x, y):
    self.x = x
    self.y = y

    out = x * y
    return out
  
  # forward 할때 저장해 놓았든 x, y를 각각 반대 방향으로 미분값과 곱해서 리턴
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

곱셈 레이어 테스트 하기

In [ ]:
# 순전파
apple = 100   # 사과 1개당 가격
apple_cnt = 2 # 사과 개수
tax = 1.1     # 소비세

# 계층은 2개
# (apple * apple_cnt) * tax

mul_apple_layer = MulLayer()
mul_tax_layer   = MulLayer()

# 순전파 수행
# 순서가 굉장히 중요합니다!!!!!
# 계획한 순서 그대로 레이어를 배치해서 연산을 해야 한다.
# 순서가 맞지 않으면 역전파 할 때 문제가 된다.

# 순전파 시에 A-B-C 순으로 계산을 했다면
# 역전파 시에는 C-B-A 순으로 역전파가 되어야 한다.

apple_price = mul_apple_layer.forward(apple, apple_cnt)
price       = mul_tax_layer.forward(apple_price, tax)

print("최종 사과의 가격 : {:.0f}".format(price))

최종 사과의 가격 : 220


In [ ]:
# 역전파 수행하기
# 제일 마지막 값에 대한 미분값 생각하기
# d돈통 / d포스기 = 1
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 전체 가격(사과 1개 가격 * 사과 개수)의 미분값 : {}".format(dapple_price))
print("사과 1개 가격에 대한 미분값 : {}".format(dapple))
print("사과 개수에 대한 미분값 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 : {}".format(dtax))

사과 전체 가격(사과 1개 가격 * 사과 개수)의 미분값 : 1.1
사과 1개 가격에 대한 미분값 : 2.2
사과 개수에 대한 미분값 : 110.00000000000001
소비세에 대한 미분값 : 200


# 2. 덧셈 계층 구현하기
* `forward` : `x + y`
* `backward` : 뒷층에서 보내진 미분값 * 1만 하면 된다.( `dout * 1` )
* 비고 : 곱셈계층과는 다르게 `forward`시에 입력된 값을 가지고 있지 않아도 된다. 역전파 시에는 들어온 미분값만 리턴하면 되니까

In [ ]:
class AddLayer:

  def __init__(self):
    # 할 거 없다.
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1

    return dx, dy

In [ ]:
# 사과와 오렌지를 장을 볼게요
apple = 100
apple_cnt = 2

orange = 150
orange_cnt = 3

tax = 1.1

In [ ]:
# 1 계층( 사과 1개 * 사과 가격, 오렌지1 * 오렌지 가격)
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()

# 2 계층 ( 사과 전체 가격 + 오렌지 전체 가격 )
add_apple_orange = AddLayer()

# 3 계층 ( 소비세 곱하기 )
mul_tax_layer = MulLayer()

In [ ]:
# 1 계층 연산
apple_price = mul_apple_layer.forward(apple, apple_cnt)
orange_price = mul_orange_layer.forward(orange, orange_cnt)

# 2 계층 연산
total_price = add_apple_orange.forward(apple_price, orange_price)

# 3 계층 연산( 소비세 연산 )
price = mul_tax_layer.forward(total_price, tax)

print("최종 가격 : {}".format(price))

최종 가격 : 715.0000000000001


In [ ]:
# 역전파
dprice = 1 # d돈통 / d포스기

# dprice / dtotal_price, dprice / tax -> ( d포스기 / d전체 가격), (d포스기 / d소비세)
dtotal_price, dtax = mul_tax_layer.backward(dprice)

# d돈통 / dapple_price = ( d돈통 / d포스기 ) * ( d포스기 / dtotal_price ) * ( dtotal_price / dapple_price )
dapple_price, dorange_price = add_apple_orange.backward(dtotal_price)

# 사과와 오렌지에 대한 각각의 미분값 구하기
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)
dorange, dorange_cnt = mul_orange_layer.backward(dorange_price)

In [ ]:
print("사과 개수에 대한 돈통의 미분값(d돈통/d사과개수) : {}".format(dapple_cnt))
print("사과 1개 가격에 대한 돈통의 미분값 (d돈통/d사과1개가격) : {}".format(dapple))

print("오렌지 개수에 대한 돈통의 미분값(d돈통/d오렌지개수) : {}".format(dorange_cnt))
print("오렌지 1개 가격에 대한 돈통의 미분값 (d돈통/d오렌지1개가격) : {}".format(dorange))

print("소비세에 대한 돈통의 미분값 (d돈통/d소비세) : {}".format(dtax))

사과 개수에 대한 돈통의 미분값(d돈통/d사과개수) : 110.00000000000001
사과 1개 가격에 대한 돈통의 미분값 (d돈통/d사과1개가격) : 2.2
오렌지 개수에 대한 돈통의 미분값(d돈통/d오렌지개수) : 165.0
오렌지 1개 가격에 대한 돈통의 미분값 (d돈통/d오렌지1개가격) : 3.3000000000000003
소비세에 대한 돈통의 미분값 (d돈통/d소비세) : 650


# 신경망 레이어 만들기
